In [1]:
# RBM : Simple NN which uses backpropogation to minimize difference between input and activation : Minimize PDF ; We dont-
# have labels to validate so backpropogate and minimize error in original data reconstruction using same weight 
# Good Expln : http://deeplearning4j.org/restrictedboltzmannmachine.html
# Math'y : http://deeplearning.net/tutorial/rbm.html
# use this to find classify unique chords  
# fancy :If you stack multiple such RBM's over each other -> DBN 


In [2]:
import collections
import math
from collections import defaultdict
import pandas as pd
import numpy as np
import scipy.sparse
import random, pickle
import re,itertools
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC


%matplotlib inline





In [4]:
chords = collections.defaultdict(list)
with open('uniqueChords.txt','r') as f :
    for idx,i in enumerate(f):
        notes=i.split()
        chords[idx]=notes


In [6]:
notelist = []
notes  = pd.read_csv('midiToNotes.txt',skiprows=2)
for n,l in zip(notes['Note/Rest'],notes['Octave']):
    notelist.append('%s%s'%(n,l))

In [7]:
# piano notes to number --> 0-87
noteValues = {'C':0,'D':2,'E':4,'F':5,'G':7,'A':9,'B':11}
accidentals = {'-':1,'#':1}

chordList = {}
for idx,notelist in chords.items():
    an = []
    for i,v in enumerate(notelist):
        k = v[0]
        c = noteValues.get(k)
        if (v.find('-')):
            c=c-1
        elif (v.find('#')):
            c=c+1 
        d = int(re.search('\d',v).group())
        c = 12*d+c
        an.append(c)
    chordList[idx] = an
    #allnotes = [v for k,v in an.items()]            

In [8]:
#one hot encoding over 88 values for each note value
stack = np.zeros((1,88))
for idx,notes in chordList.items():
    allBits = np.zeros((1,88))
    for ix,i in enumerate(notes):
        allBits[0][i] = 1
    if idx == 0 : stack = allBits
    else : stack = np.vstack((stack,allBits))
    



In [9]:
# adding some more training examples by using notes as postion in pentatonic scale
# circle of fifths : Major and Minor

'''minorPentatonica = ['C':[],
               'G':[],
               'D':[],
               'A':[''],
               'E':[],
               'B':[],
               'G-':[],
               'D-':[],
               'A-':[],
               'E-':[],
               'B-':[],
               'F':[]
              ]
              '''
def genAltered(note='C3'):
    # In case you have to convert a note (e.g. F#) into form below
    def convertSharps(note):
        pitch = ''.join([i for i in note if i.isdigit() is False])
        enharmonic = {"C#" : "D-", "D#" : "E-", "E#" : "F", "F#" : "G-", "G#" : "A-", "A#" : "B-", "B#" : "C"}
        if '#' in pitch: return enharmonic[pitch]
        return pitch
    
    # Get scale with dictionary. For example: allscales[note[:-1]]
    allscales = {
        "C"  : ["C3", "E-3", "F3", "G3", "B-3",
                "C4", "E-4", "F4", "G4", "B-4",
                "C5", "E-5", "F5", "G5", "B-5",
                "C6", "E-6", "F6", "G6", "B-6"],
        "D-" : ["D-3", "E3", "G-3", "A-3", "B3",
                "D-4", "E4", "G-4", "A-4", "B4",
                "D-5", "E5", "G-5", "A-5", "B5",
                "D-6", "E6", "G-6", "A-6", "B6"],
        "D"  : ["C3", "D3", "F3", "G3", "A3", 
                "C4", "D4", "F4", "G4", "A4", 
                "C5", "D5", "F5", "G5", "A5", 
                "C6", "D6", "F6", "G6", "A6"],
        "E-" : ["D-3", "E-3", "G-3", "A-3", "B-3",
                "D-4", "E-4", "G-4", "A-4", "B-4",
                "D-5", "E-5", "G-5", "A-5", "B-5",
                "D-6", "E-6", "G-6", "A-6", "B-6"],
        "E"  : ["D3", "E3", "G3", "A3", "B3",
                "D4", "E4", "G4", "A4", "B4",
                "D5", "E5", "G5", "A5", "B5",
                "D6", "E6", "G6", "A6", "B6"],
        "F"  : ["C3", "E-3", "F3", "A-3", "B-3",
                "C4", "E-4", "F4", "A-4", "B-4",
                "C5", "E-5", "F5", "A-5", "B-5",
                "C6", "E-6", "F6", "A-6", "B-6"],
        "G-" : ["D-3", "E3", "G-3", "A3", "B3",
                "D-4", "E4", "G-4", "A4", "B4",
                "D-5", "E5", "G-5", "A5", "B5",
                "D-6", "E6", "G-6", "A6", "B6"],
        "G"  : ["C3", "D3", "F3", "G3", "B-3",
                "C4", "D4", "F4", "G4", "B-4",
                "C5", "D5", "F5", "G5", "B-5",
                "C6", "D6", "F6", "G6", "B-6"],
        "A-" : ["D-3", "E-3", "G-3", "A-3", "B3",
                "D-4", "E-4", "G-4", "A-4", "B4",
                "D-5", "E-5", "G-5", "A-5", "B5",
                "D-6", "E-6", "G-6", "A-6", "B6"],
        "A"  : ["C3", "D3", "E3", "G3", "A3",
                "C4", "D4", "E4", "G4", "A4",
                "C5", "D5", "E5", "G5", "A5",
                "C6", "D6", "E6", "G6", "A6"],
        "B-" : ["D-3", "E-3", "F3", "A-3", "B-3",
                "D-4", "E-4", "F4", "A-4", "B-4",
                "D-5", "E-5", "F5", "A-5", "B-5",
                "D-6", "E-6", "F6", "A-6", "B-6"],
        "B"  : ["D3", "E3", "G-3", "A3", "B3",
                "D4", "E4", "G-4", "A4", "B4",
                "D5", "E5", "G-5", "A5", "B5",
                "D6", "E6", "G-6", "A6", "B6"]}
    pitch = ''.join([i for i in note if i.isdigit() is False])
    pitch = convertSharps(note) # Rm. octaveinfo, eg. G-5 --> G-, G5->G
    return allscales[pitch]

In [10]:
def quantify(note):
    notevals = {
        'C' : 0,
        'D' : 2,
        'E' : 4,
        'F' : 5,
        'G' : 7,
        'A' : 9,
        'B' : 11
    }
    quantized = 0
    octave = int(note[-1]) - 1
    for i in note[:-1]:
        if i in notevals: quantized += notevals[i]
        if i == '-': quantized -= 1
        if i == '#': quantized += 1
    quantized += 12 * octave
    return quantized

def genChordNotes(chord):
    notevect = np.zeros((1, 88))
    
    # populate with initial pitches
    for note in chord:
        notevect[0, quantify(note)] = 1
        
    # add initial pitches transposed to other octaves
    otheroctaves = range(3, 6)
    for note in chord:
        notebase = note[:-1]
        for octv in otheroctaves:
            put = bool(random.getrandbits(1)) # randomize other pitches
            if put is True:
                translated = "%s%s" % (notebase, octv)
                notevect[0, quantify(translated)] = 1
    altfreqs = defaultdict(int)
    for note in chord:
        for i in genAltered(note):
            altfreqs[i] += 1
    topnotes = [k for k, v in altfreqs.items() if v > 2] # get notes that overlap > 2 times
    for note in topnotes: # flip bits randomly from this list
        if bool(random.getrandbits(1)):
            notevect[0, quantify(note)] = 1
    
    # return the vector
    return notevect

more_x = np.zeros((1, 88))
more_y = [k for k,v in chords.items()]
for chordID, chord in chords.items():
    for j in range(50): 
        more_x = np.vstack((more_x, genChordNotes(chord)))
        more_y.append(chordID)
more_y = np.array(more_y).reshape((-1))       
print(more_x.shape)
print(more_y.shape)


(2001, 88)
(2040,)


In [11]:
tt = np.delete(more_x,0,axis=0)

In [12]:
tt.shape

(2000, 88)

In [13]:
more_x = tt

In [14]:
more_y

array([ 0,  1,  2, ..., 39, 39, 39])

In [15]:
X = np.vstack((stack,more_x))
n_samples,x = X.shape
#ydata = [k for k,v in chords.items()]
#y = np.array(ydata)
y = more_y
print(X.shape)
print(y.shape)
print(n_samples)

(2040, 88)
(2040,)
2040


In [16]:
y

array([ 0,  1,  2, ..., 39, 39, 39])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)


In [18]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100,1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100,1000]}]
scores = ['precision', 'recall']

In [43]:
#scikit
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 

Best parameters set found on development set:

{'kernel': 'rbf', 'C': 100, 'gamma': 0.001}

Grid scores on development set:

0.003 (+/-0.009) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001}
0.003 (+/-0.009) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001}
0.796 (+/-0.035) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}
0.003 (+/-0.009) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001}
0.923 (+/-0.027) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.001}
0.796 (+/-0.035) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.0001}
0.915 (+/-0.032) for {'kernel': 'rbf', 'C': 1000, 'gamma': 0.001}
0.921 (+/-0.028) for {'kernel': 'rbf', 'C': 1000, 'gamma': 0.0001}
0.915 (+/-0.020) for {'kernel': 'linear', 'C': 1}
0.915 (+/-0.021) for {'kernel': 'linear', 'C': 10}
0.914 (+/-0.025) for {'kernel': 'linear', 'C': 100}
0.912 (+/-0.026) for {'kernel': 'linear', 'C': 1000}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

            

In [47]:
with open('classifier.pickle','wb') as f :
    pickle.dump(clf.best_estimator_,f)
f.close()

In [48]:
with open('chords.pickle','wb') as f:
    pickle.dump(chords,f)
f.close()